<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_8_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1017      0 --:--:-- --:--:-- --:--:--  1021
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-8.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-8.validation.csv",header=None)
valid.head()

,0,1
0,0,scared to death
1,2,heel pain
2,5,increased heart rate
3,6,stage 1 diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-8.test.csv",header=None)
test.head()

,0,1
0,1,Abdominal rash
1,5,increase my heart rate
2,5,tachycardia
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.458528,4.777704,0.232340,11:05


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.663051,4.466008,0.260977,11:02


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.370026,4.430391,0.264903,10:58


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.314543,4.404990,0.266623,11:02


In [0]:
learn.save_encoder('pubmed_fold8_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold8_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.632095,3.486254,0.418002,01:07


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.805324,3.055176,0.479655,01:19


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.486869,2.742682,0.515413,03:39


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.202449,2.511773,0.549938,03:28
1,1.999733,2.406646,0.554871,03:29


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.915681,2.311818,0.580764,03:38
1,1.778170,2.182605,0.602959,03:28


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.718171,2.153555,0.611591,03:24
1,1.535448,2.088311,0.610358,03:27


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.508014,2.061253,0.627620,03:24
1,1.458568,2.014574,0.616523,03:37


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.412187,2.011905,0.626387,03:17
1,1.357542,1.971927,0.633785,03:36


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.347398,1.992536,0.642417,03:15
1,1.355119,2.031464,0.628853,03:22
2,1.239957,1.939848,0.639951,03:33
3,1.167052,1.964981,0.637485,03:19


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.181145,1.975162,0.637485,03:25
1,1.216739,2.012314,0.642417,03:34
2,1.122380,2.019879,0.644883,03:24
3,1.087480,1.991508,0.641184,03:26


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.083102,2.023461,0.638718,03:26
1,1.154804,2.005978,0.641184,03:50
2,1.112330,2.014726,0.652281,03:42
3,1.036921,2.025439,0.653514,03:40


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.025924,2.009679,0.654747,03:19
1,1.089301,2.064607,0.644883,03:11
2,1.039202,2.096161,0.647349,03:28
3,0.973378,2.051398,0.646116,03:33


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.989254,2.048391,0.638718,03:13
1,1.060819,2.061087,0.644883,03:35
2,1.006153,2.063295,0.649815,03:23
3,0.925766,2.077883,0.649815,03:35


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.967423,2.123240,0.635018,03:23
1,0.998328,2.120834,0.652281,03:38
2,0.997378,2.099161,0.648582,03:16
3,0.911340,2.112347,0.647349,03:29


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.940591,2.107317,0.646116,03:31
1,1.010468,2.155486,0.646116,03:22
2,0.940158,2.119152,0.649815,03:30
3,0.948017,2.169732,0.651048,03:25


In [0]:
# save the best model

learn.save_encoder('ask-8-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[34, 379, 71, 8, 8, 8, 71, 20, 498, 276, 663, 491, 584, 31, 197, 107, 76, 876, 741, 289, 117, 117, 42, 71, 49, 49, 49, 49, 621, 847, 49, 53, 53, 53, 53, 53, 60, 60, 60, 182, 71, 76, 76, 79, 79, 71, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 686, 71, 79, 80, 80, 80, 80, 189, 71, 455, 71, 71, 94, 94, 94, 94, 71, 431, 431, 431, 431, 94, 94, 94, 94, 94, 94, 289, 129, 94, 234, 71, 337, 100, 100, 100, 100, 100, 100, 71, 265, 108, 108, 419, 265, 111, 111, 111, 111, 111, 111, 111, 111, 111, 289, 71, 117, 471, 585, 125, 125, 125, 630, 630, 125, 125, 125, 125, 125, 289, 728, 127, 135, 94, 359, 741, 330, 268, 137, 71, 142, 847, 71, 71, 152, 157, 265, 270, 165, 71, 166, 779, 172, 172, 289, 172, 71, 359, 904, 182, 182, 182, 182, 182, 703, 182, 826, 183, 183, 183, 183, 183, 183, 183, 71, 71, 185, 511, 187, 187, 187, 655, 470, 190, 190, 190, 2

In [44]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

471
0.5438799076212472
